In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys
from selenium.webdriver.common.by   import  By
from selenium.webdriver.support.ui  import  WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import dill as pickle
import json

In [11]:
with open("secrets.json", 'r') as secrets:
    s = json.load(secrets)
    email = s['EMAIL']
    pw = s['PASSWORD']


In [12]:
class DairySpider:
    def __init__(self):
        self.documents = []
        self.tag = None
        self.df = None
        self.driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')
        self.login()
        
    def login(self, email=email, pw=pw, login_url='https://veganbootcamp.org/login'):
        self.driver.get(login_url)
        try:
            email_field = self.driver.find_element_by_id('login-email')
            pw_field = self.driver.find_element_by_id('login-password')
            email_field.send_keys(email)
            pw_field.send_keys(pw)
            submit_btn = self.driver.find_element_by_class_name('btn').click()
        except:
            time.sleep(2)
            self.driver.quit()
            raise Exception("Could not login")

        self.crawl(start=True)

    def crawl(self, start=False, stale=None):
        try:
            if start == True:
                course = WebDriverWait(self.driver,10).until(EC.element_to_be_clickable( (By.XPATH, "//button[@class='btn featured-course-card-btn']") ))
                course.click()
                first_track = WebDriverWait(self.driver,10).until(EC.element_to_be_clickable( (By.LINK_TEXT, "Overview of Dairy") ))
                first_track.click()
                p_tags = WebDriverWait(self.driver,10).until(lambda d: d.find_elements_by_tag_name("p"))
            
            else:
                print('looped around')
                WebDriverWait(self.driver, 10).until(EC.staleness_of(stale))
                p_tags = self.driver.find_elements_by_tag_name('p')
            
            if len(self.driver.find_elements_by_xpath("//h3[contains(text(), 'Quiz')]")) > 0:
                print('found a quiz')
                self.quit_driver()
                self.saveJson()
            else:
                course_tags = self.driver.find_elements_by_tag_name("b")
                if len(course_tags) > 0:
                    course_tag = course_tags[0].text
                    if self.tag == None:
                        self.tag = course_tag
                    if self.tag != course_tag:
                        print("Are you sure you're ready to switch course tags?", course_tag, self.tag)
                url = self.driver.current_url
                for p in p_tags:
                    doc = {}
                    doc['tag'] = self.tag
                    doc['text'] = p.text
                    print(p.text)
                    doc['description'] = 'lesson'
                    doc['url'] = url
                    self.documents.append(doc)
                quotes = self.driver.find_elements_by_tag_name("blockquote")
                for q in quotes:
                    doc = {}
                    doc['tag'] = self.tag
                    doc['text'] = q.text
                    doc['description'] = 'quote'
                    doc['url'] = url
                    self.documents.append(doc)
                next_lessons = self.driver.find_elements_by_xpath("//input[@value='Next lesson']")
                if len(next_lessons) > 0:
                    print('clicking next')
                    next_lessons[0].click()
                    self.crawl(stale=next_lessons[0])
                else:
                    print('no next')
                    self.quit_driver()
                    self.saveJson()
        except:
            self.quit_driver()
            raise Exception("Could not get all dairy info")
    
    def quit_driver(self):
        time.sleep(2)
        self.driver.quit()
            
    def saveJson(self):
        print('saving json')
        self.df = pd.DataFrame(self.documents)
        self.df.to_json(f'json_data/{self.tag}_data.json')
        

In [87]:
test = [1,2,3,4,5]
b = test.pop()
b

5

In [92]:
class Spider:
    def __init__(self, course_links=None):
        self.documents = []
        self.tag = None
        self.df = None
        self.driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')
        self.courses = None
        self.counter = 0
        self.course_links = course_links
        self.login()
        
        
    def login(self, email="amydsi.ga@gmail.com", pw="snowballpw", login_url='https://veganbootcamp.org/login'):
        self.driver.get(login_url)
        try:
            email_field = self.driver.find_element_by_id('login-email')
            pw_field = self.driver.find_element_by_id('login-password')
            email_field.send_keys(email)
            pw_field.send_keys(pw)
            submit_btn = self.driver.find_element_by_class_name('btn').click()
        except:
            time.sleep(2)
            self.driver.quit()
            raise Exception("Could not login")

        self.start_course()
        
    def start_course(self):
        try:
            WebDriverWait(self.driver,10).until(EC.element_to_be_clickable( (By.XPATH, "//button[@class='btn featured-course-card-btn']") ))
            if len(self.course_links) > 0:
                course_link = self.course_links.pop()
                print('course_link', course_link)
                course = self.driver.find_element(By.LINK_TEXT, 'Resume Course')
                self.driver.get(f'https://veganbootcamp.org/course-progress/{course_link}')
                
            else:
                course = self.driver.find_element(By.XPATH, "//input[@value='Begin Course']")
                course.click()

        except:
            self.quit_driver()
            raise Exception("Could not find a new course to start")
        
        self.crawl(course)
    
    def crawl(self, stale):
        WebDriverWait(self.driver, 10).until(EC.staleness_of(stale))
        if len(self.driver.find_elements_by_xpath("//h3[contains(text(), 'Quiz')]")) > 0:
            print('on a quiz')
            self.saveJson()
            self.dash()
     
        else:
            try:
                course_tags = self.driver.find_elements_by_tag_name("b")
                if len(course_tags) > 0:
                    course_tag = course_tags[0].text
                    if self.tag == None:
                        print('new tag', course_tag)
                        self.tag = course_tag
                    if self.tag != course_tag:
                        print("Are you sure you're ready to switch course tags?", course_tag, self.tag)
            except:
                print('no tag?', self.tag)
                
            url = self.driver.current_url
            p_tags = self.driver.find_elements_by_tag_name('p')
            for p in p_tags:
                doc = {}
                doc['tag'] = self.tag
                doc['text'] = p.text
                doc['description'] = 'lesson'
                doc['url'] = url
                self.documents.append(doc)
            quotes = self.driver.find_elements_by_tag_name("blockquote")
            for q in quotes:
                doc = {}
                doc['tag'] = self.tag
                doc['text'] = q.text
                doc['description'] = 'quote'
                doc['url'] = url
                self.documents.append(doc)
            next_lessons = self.driver.find_elements_by_xpath("//input[@value='Next lesson']")
            if len(next_lessons) > 0:
                print('clicking next')
                time.sleep(1)
                next_lessons[0].click()
                self.crawl(stale=next_lessons[0])
            else:
                print('no next')
                self.saveJson()
                self.dash()

    
    def dash(self):
        try:
            home_button = self.driver.find_element(By.XPATH, "//img[@alt='Vegan Bootcamp']")
            home_button.click()
        except:
            print('could not find the home button')
            self.driver.get("https://veganbootcamp.org/courses")
        WebDriverWait(self.driver,10).until(EC.element_to_be_clickable( (By.XPATH, "//button[@class='btn featured-course-card-btn']") ))
        self.start_course()
        
    
    def quit_driver(self):
        time.sleep(2)
        self.driver.quit()
            
    def saveJson(self):
        print('saving json for', self.tag)
        self.df = pd.DataFrame(self.documents)
        self.df.to_json(f'json_data/{self.tag}_data.json')
        self.documents = []
        self.df = None
        self.tag = None
        

In [96]:
gen_spider = Spider(['585/watch-dominion-2018', '518/can-you-love-animals-and-eat-them', '508/nature-biology-or-evolution-justifications', '708/introduction-to-activist-funding'])

course_link 708/introduction-to-activist-funding
new tag Get Activist Funding
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
on a quiz
saving json for Get Activist Funding
course_link 508/nature-biology-or-evolution-justifications
new tag Justifications
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
on a quiz
saving json for Justifications
course_link 518/can-you-love-animals-and-eat-them
new tag 30 Arguments
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
clicking next
no next
saving json for 30 Arguments
could not find the home butt

Exception: Could not find a new course to start

In [45]:
gen_spider.documents

[]

In [50]:
df = pd.read_json('json_data/Meat_data.json')
df.tail()

,tag,text,description,url
191,Meat,Here's a trick ingredient you may have never c...,lesson,https://veganbootcamp.org/course-progress/618/...
192,Meat,Nutritional quality indices show plant-based d...,lesson,https://veganbootcamp.org/course-progress/619/...
193,Meat,A question so common that it's become a joke i...,lesson,https://veganbootcamp.org/course-progress/619/...
194,Meat,"When it comes to getting plant-based protein, ...",lesson,https://veganbootcamp.org/course-progress/620/...
195,Meat,If there are other reasons you have to avoid s...,lesson,https://veganbootcamp.org/course-progress/620/...


In [54]:
fish = pd.read_json('json_data/Fish_data.json')
fish.shape
fish.tail()

,tag,text,description,url
38,Fish,If there's one sea-based charity vegans should...,lesson,https://veganbootcamp.org/course-progress/575/...
39,Fish,Although not strictly a vegan outreach charity...,lesson,https://veganbootcamp.org/course-progress/575/...
40,Fish,Sea Shepherd is a global organization with man...,lesson,https://veganbootcamp.org/course-progress/575/...
41,Fish,"To attract female fish, the Japanese puffer fi...",lesson,https://veganbootcamp.org/course-progress/576/...
42,Fish,Sushi cravings are real. YouTuber NikkiVegan t...,lesson,https://veganbootcamp.org/course-progress/577/...


In [15]:
dairy_spider = Spider()

Many people believe that cow's milk is essential in the human diet after years of the dairy industry promoting its product as healthy. However, there are a number of health risks associated with consuming it. We'll go over some of those risks in the next section.
"Cow’s milk is promoted as natural, wholesome and healthy–it is none of these things."
The saturated fat, animal protein, cholesterol, and hormones that it contains are linked to a wide range of illnesses and diseases. Aggressive marketing of milk and dairy products has resulted in confusion–people simply don't know who to believe.
Most people in the world—over 70 percent—are lactose intolerant. No other animal on the planet continues to drink milk after weaning, let alone the milk of a pregnant mother from another species. It begs the question, are we asking for trouble?
Humans have been drinking milk for only 6,000-8,000 years. This might sound like a very long time but in evolutionary terms, it is very recent history.
If th

Cows on organic dairy farms can be kept in crowded sheds, mired in their own waste, much like cows on factory dairy farms. They, too, are artificially impregnated every year, and their calves are taken from them soon after birth. Cows on organic farms often aren’t given antibiotics—even when they’re sick or when their udders become infected, something that happens often—because medicated animals lose their “organic” status. (source)
Unfortunately, when it comes to local, the farmers still have to make a profit from said cows to make a living while competing with bigger farms, which means cutting costs like with any other "product." Local or not, cows still need to be forcibly impregnated to produce milk, still die to be sold for meat before they get close to living a full life, and male calves still get killed for veal production as they are useless to dairy farmers.
A global analysis conducted by Sentience Institute suggests over 90% of farmed animals worldwide live on factory farms. 

Lactose intolerance should not be confused with cow's milk allergy – they are entirely different. Cow's milk allergy is where the immune system reacts to proteins found in the milk wheras lactose intolerance is when the body cannot digest lactose – the sugar in milk.
Those countries which consume the most dairy foods and animal protein have the highest hip fracture rates – dairy products don't protect our bones. Despite what the dairy industry says, cow's milk is not the best source of calcium and your bone health would benefit enormously by switching to plant-based sources.
In a study published in the British Medical Journal, it was suggested that it is time we revised our calcium recommendations for young people and changed our assumptions about the role of calcium, milk and other dairy products play in the bone health of children and adolescents. Physical (especially weight-bearing) exercise is the most critical factor for maintaining healthy bones, followed by improving the diet an

Rice milk is less likely to cause food allergies compared to any other milk because it is nut and gluten-free. It has a naturally sweet taste and can be used for cooking and baking. Rice milk is extremely low in calories, which are mostly from carbohydrates, and it has very little protein or fat. Since rice milk is a rather thin milk, it is less suitable for coffee.
Of the non-dairy milks, soya milk is the most nutritionally similar to cow’s milk and is the most popular milk alternative so far. It is moderate in calories and is a good source of protein and calcium. Soya milk contains compounds called isoflavones and phytosterols, that can possibly lower the risk of cancer, cardiovascular disease and osteoporosis. It is suitable for a wide range of purposes and can be used without restriction for cooking, baking, or in coffee, and it can even be foamed. As with cow’s milk, soya milk is now available in various flavours, such as vanilla, chocolate, or banana.
Spelt milk is moderate in ca

In [17]:
dairy_spider.df

,tag,text,description,url
0,Dairy Milk,Many people believe that cow's milk is essenti...,lesson,https://veganbootcamp.org/course-progress/497/...
1,Dairy Milk,"""Cow’s milk is promoted as natural, wholesome ...",lesson,https://veganbootcamp.org/course-progress/497/...
2,Dairy Milk,"The saturated fat, animal protein, cholesterol...",lesson,https://veganbootcamp.org/course-progress/497/...
3,Dairy Milk,Most people in the world—over 70 percent—are l...,lesson,https://veganbootcamp.org/course-progress/497/...
4,Dairy Milk,"Humans have been drinking milk for only 6,000-...",lesson,https://veganbootcamp.org/course-progress/497/...
...,...,...,...,...
92,Dairy Milk,Almond milk requires more water to produce tha...,lesson,https://veganbootcamp.org/course-progress/502/...
93,Dairy Milk,People tend to underestimate the greenhouse ga...,lesson,https://veganbootcamp.org/course-progress/502/...
94,Dairy Milk,"However, this information spurs more questions...",lesson,https://veganbootcamp.org/course-progress/502/...
95,Dairy Milk,Find more cute videos and gif's of happy cows ...,lesson,https://veganbootcamp.org/course-progress/704/...


In [23]:
jsdf = pd.read_json('./json_data/Dairy_Milk_data.json')

In [25]:
jsdf[jsdf['description']=='quote']

,tag,text,description,url
12,Dairy Milk,"""Cow’s milk is promoted as natural, wholesome ...",quote,https://veganbootcamp.org/course-progress/497/...


In [ ]:



driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')
driver.get(login_url)

try:
    email = driver.find_element_by_id('login-email')
    pw = driver.find_element_by_id('login-password')
    email.send_keys('amydsi.ga@gmail.com')
    pw.send_keys('snowballpw')
    submit_btn = driver.find_element_by_class_name("btn").click()
    print(submit_btn)
except:
    print('hey')
    
#     button.click()
#     time.sleep(10)
# except Exception as e:
#     print(e)
#     time.sleep(5)
#     driver.quit()
# finally:
#     driver.quit()
# time.sleep(2)

In [ ]:
url = 'https://veganbootcamp.org/course-progress/497/overview-of-dairy'
headers = {
    'authority': 'veganbootcamp.org',
    'cache-control': 'no-cache',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'image',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
    'cookie': 'vbcommunity_k=; googtrans=/en/en; _ga=GA1.2.1818705141.1621926329; _fbp=fb.1.1622009394616.990287943; vbcommunity_u=4947; _gid=GA1.2.443184538.1622230205; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlNVK29Oa0tHQkF3QWJsSUU5NnQva0E9PSIsInZhbHVlIjoidHZ6ck9BS3FYaHdIVkMxYlNtRGlqVUpqeWc2WXhBWkpYOGpQVm5zVHNycnkwWFNQNm5CdWJQMlR6YjJpYjRsYkJaOHM5VEttMUtJTzd6UnVqOW9Vb2JLQmNkZGR0c2dBR1VrQUR5KzNrSU9nRkFncjVlNlIva01aayswTEFwcUgzdHdtMVQ1Yk1RT093N0pDa0VIeWJaNFdYL2E1S1Uzd0pySm5hamtCcnVZPSIsIm1hYyI6ImE5Zjk3OWMzNTc1YWRjMTJhZmIwYWFlMjgyNzVmYTZhN2U4MDI2NTNmMDAyNDgyNzhlNDc0NjRmNmEyNTlkMzMifQ%3D%3D; vbcommunity_sid=02e77f07321a010ee42fdbee9a8bb247; _gat_gtag_UA_150398169_1=1; XSRF-TOKEN=eyJpdiI6IlhYVlJ5NkwzcWtnTG9PZDVqVFpVekE9PSIsInZhbHVlIjoiL2gxOVhzeDJ4OS9RcHQ3b214ZWl2YXRIYXAzdm1hZnZKKzBMSlhTOGpLV2J0eTdYeFFESCs1NnpacUVlMzF2ZCIsIm1hYyI6IjkzOTc0Mjg0NzU5M2E5NWY2NjVmN2YyYmIwNjdiMjllYWU3MGMyNmQxYzdmNWEwYjExYzkwMjM1ZmY0M2MzYTIifQ%3D%3D; vegan_bootcamp_session=eyJpdiI6IitiOVZiNjBHVVJBTC8vd0IzSGptUWc9PSIsInZhbHVlIjoiaTU3TjdnckhWeDBEZkZWNithVThINmJQa0V0RHl6VlcrSXNqTVJ0Y29MeFY2YU1qYm9FTldHbFU2ckcvbnU1ZyIsIm1hYyI6IjU0MTU3MDM4ZTNlOTE5MzZjMzE5MjczNWM5ZDMwNzM2NjY3M2Q4MTI3MThjZmIxMzA0YjBhOTVmMGRkN2MyMTEifQ%3D%3D',
    'Referer': 'https://veganbootcamp.org/courses',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'x-client-data': 'CIu2yQEIprbJAQjBtskBCKmdygEIqZ3LAQiioMsBCNzyywEIqPPLAQ==',
    'referer': 'https://veganbootcamp.org/courses',
    'Origin': 'https://veganbootcamp.org',
    'pragma': 'no-cache',
}

dairy_spider = Spider(url, headers)
dairy_spider.crawl()

In [ ]:
url = 'https://veganbootcamp.org/course-progress/497/overview-of-dairy'

headers = {
    'authority': 'veganbootcamp.org',
    'cache-control': 'no-cache',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'image',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
    'cookie': 'vbcommunity_k=; googtrans=/en/en; _ga=GA1.2.1818705141.1621926329; _fbp=fb.1.1622009394616.990287943; vbcommunity_u=4947; _gid=GA1.2.443184538.1622230205; remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6IlNVK29Oa0tHQkF3QWJsSUU5NnQva0E9PSIsInZhbHVlIjoidHZ6ck9BS3FYaHdIVkMxYlNtRGlqVUpqeWc2WXhBWkpYOGpQVm5zVHNycnkwWFNQNm5CdWJQMlR6YjJpYjRsYkJaOHM5VEttMUtJTzd6UnVqOW9Vb2JLQmNkZGR0c2dBR1VrQUR5KzNrSU9nRkFncjVlNlIva01aayswTEFwcUgzdHdtMVQ1Yk1RT093N0pDa0VIeWJaNFdYL2E1S1Uzd0pySm5hamtCcnVZPSIsIm1hYyI6ImE5Zjk3OWMzNTc1YWRjMTJhZmIwYWFlMjgyNzVmYTZhN2U4MDI2NTNmMDAyNDgyNzhlNDc0NjRmNmEyNTlkMzMifQ%3D%3D; vbcommunity_sid=02e77f07321a010ee42fdbee9a8bb247; _gat_gtag_UA_150398169_1=1; XSRF-TOKEN=eyJpdiI6IlhYVlJ5NkwzcWtnTG9PZDVqVFpVekE9PSIsInZhbHVlIjoiL2gxOVhzeDJ4OS9RcHQ3b214ZWl2YXRIYXAzdm1hZnZKKzBMSlhTOGpLV2J0eTdYeFFESCs1NnpacUVlMzF2ZCIsIm1hYyI6IjkzOTc0Mjg0NzU5M2E5NWY2NjVmN2YyYmIwNjdiMjllYWU3MGMyNmQxYzdmNWEwYjExYzkwMjM1ZmY0M2MzYTIifQ%3D%3D; vegan_bootcamp_session=eyJpdiI6IitiOVZiNjBHVVJBTC8vd0IzSGptUWc9PSIsInZhbHVlIjoiaTU3TjdnckhWeDBEZkZWNithVThINmJQa0V0RHl6VlcrSXNqTVJ0Y29MeFY2YU1qYm9FTldHbFU2ckcvbnU1ZyIsIm1hYyI6IjU0MTU3MDM4ZTNlOTE5MzZjMzE5MjczNWM5ZDMwNzM2NjY3M2Q4MTI3MThjZmIxMzA0YjBhOTVmMGRkN2MyMTEifQ%3D%3D',
    'Referer': 'https://veganbootcamp.org/courses',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'x-client-data': 'CIu2yQEIprbJAQjBtskBCKmdygEIqZ3LAQiioMsBCNzyywEIqPPLAQ==',
    'referer': 'https://veganbootcamp.org/courses',
    'Origin': 'https://veganbootcamp.org',
    'pragma': 'no-cache',
}

response = requests.get(url, headers=headers)
response

In [ ]:
dairy_spider.df

In [ ]:
soup = BeautifulSoup(response.content, 'lxml')

In [ ]:
tag = soup.find('b')
tag = tag.text
tag

In [ ]:
course_info = soup.find("div", {"class": "gutenberg__content"})
documents = []
for p in course_info.find_all('p'):
    doc = {}
    doc['tag'] = tag
    doc['text'] = p.text
    doc['description'] = 'lesson'
    doc['url'] = url
    documents.append(doc)

In [ ]:
for quote in course_info.find_all('blockquote'):
    doc = {}
    doc['tag'] = tag
    doc['text'] = quote.text
    doc['description'] = 'quote'
    doc['url'] = url
    documents.append(doc)

In [ ]:
df = pd.DataFrame(documents)

In [ ]:
df.to_json('data.json')

In [ ]:
df

In [ ]:
  soup = BeautifulSoup(response.content, 'lxml')
#         tag = soup.find('b').text
#         if self.tag == None:
#             self.tag = tag
#         if tag != self.tag:
#             raise Exception(f'Whoooah are you ready to switch tags? {tag} {self.tag}')
#         course_info = soup.find("div", {"class": "gutenberg__content"})
#         # content in p tags, lesson content
#         for p in course_info.find_all('p'):
#             doc = {}
#             doc['tag'] = tag
#             doc['text'] = p.text
#             doc['description'] = 'lesson'
#             doc['url'] = url
#             self.documents.append(doc)
#         # quotes in blockquotes
#         for quote in course_info.find_all('blockquote'):
#             doc = {}
#             doc['tag'] = tag
#             doc['text'] = quote.text
#             doc['description'] = 'quote'
#             doc['url'] = url
#             self.documents.append(doc)
#         self.saveJson()